This is an auxiliary script to the main scripts calculating the consumption intensity for the products in each of the different Baskets of Products (BoPs) within the european averaged citizen consumption footprint by country project.

It contains four functions: *read_file*, *filter_file*, *inventory_dtype* and *population*, which are to be use for loading additional information from sources different from [PRODCOM](https://ec.europa.eu/eurostat/web/prodcom/data/database) and [COMEXT](https://ec.europa.eu/eurostat/web/international-trade-in-goods/data/database) for which the pyhton package [*eurostat*](https://pypi.org/project/eurostat/) will be used. 

- *read_file*, *filter_file* and *inventory_dtype* are focused on excel and csv files.
- *population* extracts information on population per country and year from EUROSTAT.


### Loading packages

In addition to Python 3.3 built-in packages, some other specific packages, such as *pandas* or *numpy* will have to be loaded too. Besides, although this can be modified anytime, the functions have been created assuming there will be two folders, *input* and *outputs* in the same directory where the present script is, containing respectively the input files to be read and the future output files resulting from the application of this script function's on input files. 

In [1]:
import os
import sys
import pandas as pd
from collections import Counter
import itertools
from itertools import compress
from itertools import product
import numpy as np
import eurostat as estat
import math

INPUTS_FOLDER = "../inputs"
OUTPUTS_FOLDER = "../outputs"


In [3]:
def read_file(file_name, file_folder, firstrow, sheetname, data_type_dict):
    
    """
    Reads the excel file containing the BoPs information.
    
    :param file_name : name of the file including extension.
    :type file_name : str.
    :param file_folder : folder where the file is.
    :type file_folder : str.
    :param firstrow :  first row with data counting from the top, being the first one row number 0.
    :type firstrow : int.
    :param sheetname : name of the sheet of interest within the document. It can also be a number, beginning with 0 for the 1st sheet.
    :type sheetname : str/int.
    :param data_type_dict : the data type of each of the columns in the file.
    :type data_type_dict : dict.
    :return : file content.
    :type return : data frame.
    
    """
    
    file_path = os.path.join(file_folder, file_name)
        
    if file_name.endswith((".xlsx", ".xls")):

        df = pd.read_excel(file_path, dtype = data_type_dict, sheet_name = sheetname, skiprows = list(range(firstrow-1)))
        
        
        
    elif file_name.endswith(".csv"):
        
        df = pd.read_csv(file_path, dtype = data_type_dict, skiprows = list(range(firstrow-1)), encoding = "latin-1")
        
       
        
    else:
        
        print("Wrong file extension")
        
        return
        
   
    #Replace nan strings by nan's numpy    
    df = df.replace(["nan", "NaN"], np.nan)
    #Delete all columns where all rows are nan
    df = df.dropna(how = 'all')
    #Delete all rows where all columns are nan
    df = df.dropna(axis = "columns", how = 'all')
    
    return(df)
    


In [18]:

def filter_file(items, input_df):
    
    """
    Finds the column containing the items and extracts the rows where these are.
    
    :param items : elements of interst. For the "Upscale.xlsx" these could be either products groups, basket of product or representative products.
    :type items : list.
    :param input_df : contains the information.
    :type ups_df : data frame.
    :return : filtered information.
    :type return : data frame.
    
    """
    #List all columns
    input_cols = input_df.columns
    
    #Creation of a list
    filtered_cols = []
    
    # It filters the file according to the first column where the item is found. 
    # Select only the FIRST column, beginning from the left, that contains the items of interest.
                          
    for col in input_cols:
        
        #From all the items demanded, it checks their existence in the column
        existing = [i for i in items if i in input_df[col].tolist()]
        #From all the items demanded, it checks their existence in the column
        missing = [i for i in items if i not in input_df[col].tolist()]
        
        #If all the items are in the column
        if len(existing) == len(items):
            
            input_filtered = input_df[input_df[col].isin(items)]
            
            #Each column that has been checked is added, given the condition that all items are in the column
            filtered_cols.append(col)
            
            break
            
        #If some items are in the colum
        elif len(missing) < len(items):
            
            print("Some items were found in {}".format(col))
            
        #TODO check with Laura
        else:
            
            continue
            
    print("All the provided items are in columns: {}".format(filtered_cols))
            
    # It makes the list of unique values in the list of filtered_cols - This could be deleted
    unique_filtered_cols = list(set(filtered_cols))
            
    return(input_filtered)

In [20]:
def inventory_dtype():
    
    dtype = {"prod_codes" : str, "EXP_COEF" : float, "IMP_COEF" : float, "PROD_COEF" : float, "components" : str,
                                              "split" : float, "DC_COEF" : str, "DC_PRODS" : str, "FBS_UPS" : float, "FAO_group":str}
    
    return(dtype)

In [22]:
def population(start_year, end_year):
    
    """
    Retrieves information on the population of all european countries in the selected years from EUROSTAT.
    
    :param start_year : first year of the period of interest.
    :type start_year : str.
    :param end_year : last year of the period of interest.
    :type end_year : str.
    
    """
    # It gets dataframe from Eurostat ()
    pop_df = estat.get_data_df("demo_r_d2jan", flags=False)
    
    #Defined columns
    cols= pop_df.columns
    #Filters total sex
    pop_df = pop_df[pop_df["sex"] == "T"]
    #Filters total age
    pop_df = pop_df[pop_df["age"] == "TOTAL"]
    #Range for time period
    years = list(range(int(start_year), (int(end_year)+1)))
    
    #Check that years are not in columns to inform about missing years data - variable could be called y = missing years
    missing_y = [y for y in years if y not in cols]
    
    if missing_y:
        print("These years are not in the database: ", missing_y)
        
    #When the years are in the dataframe
    new_years = [y for y in years if y in cols]
    #Creation of a list columns with the indicated information + the listed years
    new_cols = ["unit", "sex", "age", "geo\\time"] + new_years
    #Selection of the abovecreated list from the entire dataframe
    pop_df = pop_df[new_cols]
    #List of the codes for countries
    area_shorthand = pop_df["geo\\time"].tolist()
    
    country_dict = {'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria',
 'CY': 'Cyprus', 'CZ': 'Czechia', 'DE': 'Germany', 'DK': 'Denmark', 'EE': 'Estonia', 'EL': 'Greece', 'ES': 'Spain', 'FI': 'Finland',
 'FR': 'France', 'HR': 'Croatia', 'HU': 'Hungary', 'IE': 'Ireland', 'IT': 'Italy', 'LT': 'Lithuania','LU': 'Luxemburg', 'LV': 'Latvia',
    'MT': 'Malta', 'NL': 'Netherlands', 'PL': 'Poland', "UK" : "United Kingdom", 'PT': 'Portugal', 'RO': 'Romania', 'SE': 'Sweden', 'SI': 'Slovenia', 'SK': 'Slovakia'}
    #Filter all the country codes in a list of those countries in country_dict for EU-28    
    country_shorthand = [a for a in area_shorthand if a in country_dict.keys()]
    #Filter the dataframe
    pop_df = pop_df[pop_df["geo\\time"].isin(country_shorthand)]
    #List the countries
    coun_sh_col = pop_df["geo\\time"].tolist()
    #Assign the country name to the codes according to the dictionary
    country_names = [country_dict[csh] for csh in coun_sh_col]
    #Creation of a column with the names of the countries. This is done to enhance the link it with prodcom and comext and fao (harmonization)
    pop_df["DECL"] = country_names
    #Delete useless columns, remaining only population data and country information
    pop_df = pop_df.drop(["unit", "sex", "age", "geo\\time"], axis = 1)
    #Sort by alphabetical order according to country name
    pop_df.sort_values(by = ["DECL"])
    #Years (which are now colums) are transformed into a unique column + incl. Change names of columns
    pop_df = pop_df.set_index(["DECL"]).stack().reset_index("DECL").reset_index().rename(columns = {"index" : "Year", 0 : "Population"})
    
    # Calculate EU28 population by adding up all the countries populations for each year.
    #Aggregate data by year 
    pop_eu28 = pop_df.groupby("Year").sum()
    #Since the previous function changes columns into indexes, we need this function to transform it back to column
    pop_eu28.reset_index(inplace = True)
    #In the decl column, add the country EU28 - Check the output of this
    pop_eu28["DECL"] = ["Eu28Intra"] * len(years)
    #Add EU28 datafram to pop_df
    pop_df = pd.concat([pop_df, pop_eu28], sort = True)
    
    return(pop_df)